# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, Sugar, \
                        MolecularSystemAnimationParams, Vector2, Vector3, Quaternion, MovieMaker
import os

be = BioExplorer('localhost:5000')
status = be.reset_scene()
status = be.start_model_loading_transaction()

### Resources

In [ ]:
resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
rna_folder = os.path.join(resource_folder, 'rna')
membrane_folder = os.path.join(pdb_folder, 'membrane')

glucose_path = os.path.join(pdb_folder, 'glucose.pdb')

immune_folder = os.path.join(pdb_folder, 'immune')
lactoferrin_path = os.path.join(immune_folder, '1b0l.pdb')
defensin_path = os.path.join(immune_folder, '1ijv.pdb')

glycan_folder = os.path.join(pdb_folder, 'glycans')
complex_folder = os.path.join(glycan_folder, 'complex')
high_mannose_folder = os.path.join(glycan_folder, 'high-mannose')
hybrid_folder = os.path.join(glycan_folder, 'hybrid')
o_glycan_folder = os.path.join(glycan_folder, 'o-glycan')

complex_paths = [
    os.path.join(complex_folder, '33.pdb'),
    os.path.join(complex_folder, '34.pdb'),
    os.path.join(complex_folder, '35.pdb'),
    os.path.join(complex_folder, '36.pdb')
]
high_mannose_paths = [
    os.path.join(high_mannose_folder, '1.pdb'),
    os.path.join(high_mannose_folder, '2.pdb'),
    os.path.join(high_mannose_folder, '3.pdb'),
    os.path.join(high_mannose_folder, '4.pdb')
]
hybrid_paths = [os.path.join(hybrid_folder, '24.pdb')]
o_glycan_paths = [os.path.join(o_glycan_folder, '12.pdb')]

### Configuration

In [ ]:
# Scene
scene_size = Vector3(250.0, 250.0, 250.0)

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.protein_representation.ATOMS_AND_STICKS
glycan_radius_multiplier = 1.0
glycan_representation = be.protein_representation.ATOMS_AND_STICKS
protein_load_hydrogen = False

# Viruses
add_rna = False
add_glycans = True

## Camera

In [ ]:
status = be.core_api().set_camera(
    orientation=[-0.008, 0.391, -0.084, 0.916],
    position=[11.029, -67.295, 35.860],
    target=[-27.555, -64.584, -1.414]
)

## Scene description

### SARS-COV-2 Coronavirus

In [ ]:
be.add_sars_cov_2(
    name='sars-cov-2', resource_folder=resource_folder,
    representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    open_protein_s_indices=[0],
    position=Vector3(-48, -4.0, -7.0), 
    add_glycans=add_glycans,
    add_rna_sequence=add_rna
)

### Host cell

In [ ]:
name='Cell'

# ACE2 receptor definition
ace2_receptor = Protein(
    name=name + '_' + be.NAME_RECEPTOR,
    source=os.path.join(pdb_folder, '6m18.pdb'),
    occurrences=5,
    animation_params=MolecularSystemAnimationParams(0, 1, 0.025, 2, 0.025),
    transmembrane_params=Vector2(-8.0, 2.0)
)

# Membrane definition
size = Vector3(scene_size.x, 80.0, scene_size.z)
membrane = Membrane(
    lipid_sources=[os.path.join(membrane_folder, 'popc.pdb')],
    animation_params=MolecularSystemAnimationParams(1, 3, 0.25, 4, 0.25)
)

# Cell definition
cell = Cell(
    name=name,
    shape=be.assembly_shape.SINUSOID, shape_params=size,
    membrane=membrane, proteins=[ace2_receptor])

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    position=Vector3(-7, -82.5, -3.0)
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.protein_color_scheme.CHAINS, 'OrRd_r', 7)

In [ ]:
be.add_multiple_glycans(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    atom_radius_multiplier=glycan_radius_multiplier,
    representation=glycan_representation, assembly_name=name, 
    glycan_type=be.NAME_GLYCAN_COMPLEX,
    protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
    indices=[53, 90, 103, 322, 432, 690])

be.add_multiple_glycans(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    atom_radius_multiplier=glycan_radius_multiplier,
    representation=glycan_representation, assembly_name=name,
    glycan_type=be.NAME_GLYCAN_HYBRID,
    protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
    indices=[546])

indices = [[155, Quaternion(0.0, 0.707, 0.0, 0.707)],
           [730, Quaternion(0.0, 0.707, 0.0, 0.707)]]
for index in indices:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
    o_glycan = Sugar(
        assembly_name=name, name=o_glycan_name, 
        source=o_glycan_paths[0],
        protein_name=name + '_' + be.NAME_RECEPTOR, 
        atom_radius_multiplier=glycan_radius_multiplier,
        representation=glycan_representation,
        chain_ids=[2, 4], site_indices=[index[0]], 
        rotation=index[1])
    be.add_sugar(o_glycan)

## Materials

In [ ]:
be.apply_default_color_scheme(be.SHADING_MODE_BASIC)

## Functional regions

### Spike

In [ ]:
# RGB color palette for amino acid indices
grey = [0.2, 0.2, 0.2]
dark_green = [0.0, 1.0, 0.0]
light_green = [0.0, 1.0, 0.0]
red = [1.0, 0.0, 0.0]
green = [0.0, 1.0, 0.0]
blue = [0.0, 0.0, 1.0]
yellow = [1.0, 1.0, 0.0]
cyan = [1.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey ], [   16, grey], [  306, grey], [  330, grey], [438, dark_green], 
    [ 507, green], [  522, grey], [  816, grey ], [  835, grey ], [908, grey],
    [ 986, grey ], [ 1076, grey], [ 1274, grey], [ 2000, grey ]
]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name='sars-cov-2', name='sars-cov-2_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.protein_color_scheme.CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name='sars-cov-2', name='sars-cov-2_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2]
)

### Receptor

In [ ]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_blue = [0.0, 0.0, 0.5]
light_blue = [0.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey      ], [   5, grey],  [  30, light_blue],  [  41, dark_blue],
    [  82, light_blue], [  84, grey],  [ 353, grey      ],  [ 357, grey     ],
    [ 800, grey      ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
name = 'Cell'
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.protein_color_scheme.CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[4])

## Rendering settings

In [ ]:
status = be.set_rendering_quality(be.rendering_quality.HIGH)

In [ ]:
be.commit_model_loading_transaction()

## Snapshots

In [ ]:
mm = MovieMaker(be)
output_folder = '/tmp'
mm.create_snapshot(
    renderer='advanced', show_progress=True,
    path=output_folder, base_name='protein_s_and_ace2_receptor',
    size=[512, 512], samples_per_pixel=64)